In [1]:
import os
import pandas as pd
import numpy as np

from utils import utils
import frontmatter
import re
from mdutils import fileutils

In [3]:
dm = utils.load_and_backup_dm("../EL.data.model.csv", "../backups/")

In [15]:
data_model.rename(
    {"Attribute": "Key", "Description": "Key Description"}, axis=1, inplace=True
)

data_model = data_model[["Key", "Key Description", "Type", "Source", "Module"]]

data_model.to_csv("../_data/FullTable.csv", index=False)

post = frontmatter.load("../template_page_template.md")
post.metadata["title"] = re.sub("([A-Z]+)|_", r" \1", "FullTable").title()
post.metadata["permalink"] = f'docs/{post.metadata["title"]}.html'
post.metadata["parent"] = "Full Table"
post.content = (
    "{% assign mydata=site.data."
    + "fullTable"
    + " %} \n{: .note-title } \n"
    + f">{post.metadata['title']}\n"
    + ">\n"
    + f">Full Table of Terms ELITE portal\n"
    + post.content
)

# create directory for the moduel if not exist
if not os.path.exists(f"../docs/Full Table/"):
    os.mkdir(f"../docs/Full Table/")

    # create a module page
    module = fileutils.MarkDownFile(f"../docs/FullTable/FullTable")

    module.append_end(
        f"--- \nlayout: page \ntitle: Full Table \nhas_children: true \nnav_order: 2 \npermalink: FullTable.html \n---"
    )
# create file
file = fileutils.MarkDownFile(f"../docs/FullTable/FullTable")
# add content to the file
file.append_end(frontmatter.dumps(post))

In [7]:
# def create_term_df(data_model, term):
"""Creates the term data frame csv to populate tables in the website.

Args:
    data_model (object): full data model
    term (string): attribute term to look up

Returns:
    object: data frame object
"""

term = "countryCode"
df = dm.copy()
if "Template" in data_model.query("Attribute == @term")["Module"].values:
    # generate csv for template
    depends_on = get_template_keys(data_model, term)

    # filter out attributes from data model table
    df = data_model.loc[data_model["Attribute"].isin(depends_on),]

    # # filter out valid values from data model table. data_model uses Parent to distinguish types of attributes
    # df = df.loc[df["Parent"] != "validValue",]

    df = df[
        [
            "Attribute",
            "Description",
            "Type",
            "Valid Values",
            "DependsOn",
            "Required",
            "Source",
            "Module",
        ]
    ].reset_index(drop=True)

    df.rename(
        columns={"Attribute": "Key", "Description": "Key Description"}, inplace=True
    )
elif term == "countryCode":
    df = pd.read_excel(
        io="http://wits.worldbank.org/data/public/WITSCountryProfile-Country_Indicator_ProductMetada-en.xlsx",
        sheet_name="Country-Metadata",
    )

    df = df[
        ["Country Code", "Country Name", "Country ISO3", "Long Name", "Region"]
    ].rename({"Country Code": "Key", "Country Name": "Key Description"}, axis=1)

    df["Source"] = "https://wits.worldbank.org/countryprofile/metadata/en/country/all"
    df["Module"] = "Metadata"
    df["Type"] = "Numeric"

else:
    # prohibit = ["validValue", "BaseAnnotation", "Other"]
    df = data_model.loc[
        (~data_model["Valid Values"].isna()) | (~data_model["DependsOn"].isna()),
    ]

    df = df.loc[(df["Attribute"] == term) & (data_model["Parent"] != "validValue"),][
        ["Attribute", "Valid Values", "DependsOn", "Type", "Module"]
    ]

    # generate csv for term
    df = (
        df.drop(columns=["Attribute", "DependsOn"])
        .set_index(["Type", "Module"])
        .apply(lambda x: x.str.split(",").explode())
        .reset_index()
    )

    # add columns
    df.rename(columns={"Valid Values": "Key"}, inplace=True)
    df = df.assign(**dict([(_, None) for _ in ["Key Description", "Source"]]))
    df = df[["Key", "Key Description", "Type", "Source", "Module"]]

# return df

In [8]:
df

,Key,Key Description,Country ISO3,Long Name,Region,Source,Module,Type
0,004,Afghanistan,AFG,Islamic State of Afghanistan,South Asia,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
1,008,Albania,ALB,Republic of Albania,Europe & Central Asia,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
2,012,Algeria,DZA,People's Democratic Republic of Algeria,Middle East & North Africa,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
3,016,American Samoa,ASM,American Samoa,East Asia & Pacific,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
4,020,Andorra,AND,NaN,NaN,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
...,...,...,...,...,...,...,...,...
264,887,Yemen,YEM,Republic of Yemen,Middle East & North Africa,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
265,720,Yemen Democratic,YDR,NaN,NaN,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
266,890,"Yugoslavia, FR (Serbia/Montene",YUG,NaN,NaN,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
267,894,Zambia,ZMB,Republic of Zambia,Sub-Saharan Africa,https://wits.worldbank.org/countryprofile/meta...,Metadata,Numeric
